<a href="https://colab.research.google.com/github/leolaipelt/EarthEnginePythonBasico/blob/master/7_Usando_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Usando Landsat

O catálogo de imagens do EE possui todas as coleções dos produtos Landsat. <br>

Landsat é um programa da USGS em parceria com a NASA para mapeamento digital da superfície da Terra. Atualmente, a resolução das imagens são de 30 metros, com intervalos de tempos de 15 dias.

### Produtos Landsats

* Reflectância da Superfície (SR)
* Correção da reflectância no Topo da Superfície (TOA)
* Produtos já calculados de NDVI, EVI, índices de Vegetação...

### Satélites 

* Landsat 8 (2013 - atual)
* Landsat 7 (1999 - atual)**
* Landsat 5 (1984 - 2012)
* Landsat 4 (1982 - 1993)
* Landsat 1-5 (1972 - 1999)

 ** *O Landsat 7 apresentou falhas após o dia 31 de Maio de 2003*
 
## Outras Informações

As coleções Landsat se dividem em 3 categorias:

* Tier 1 (T1) - Atende a padrões de qualidade radiométrica e geométrica pré-determinados
* Tier 2 (T2) - Não atende aos padrões de T1
* Real Time (RT) - Imagem não passou por pré-processamento (tempo médio de 1 mês após aquisição pelo satélite)
 
#### Vamos Conferir como utilizar essas bibliotecas no EE!

In [0]:
!pip install earthengine-api

In [0]:
import ee; ee.Authenticate()

In [0]:
#Primeiro, vamos colocar nosso código de visualização!
import folium

def add_ee_layer(self,imagem,visParams, nome):
  mapID = ee.Image(imagem).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles ="https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Nosso Mapinha do Curso de Python",
    name = nome,
    overlay = True,
    control = True
  ).add_to(self)
  
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

In [0]:
#Iniciando

ee.Initialize()

#Importando uma coleção do Landsat 8 (SR)

start_date=ee.Date('2019-01-01')
end_date=ee.Date('2019-08-30')

collection = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                        .filterDate(start_date, end_date)
                        .select(["B1","B2","B3","B4","B5","B6","B7","B10","pixel_qa"])) 


In [0]:
#Vamos criar uma mascara de núvens porque nós não gostamos delas!

#Definir Bandas
bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7']


def maskL8sr(image):
  #Bits 3 e 5 são sombra de nuvem e nuvem, respectivamente
  cloudShadowBitMask = ee.Number(2).pow(3).int() 
  cloudsBitMask = ee.Number(2).pow(5).int()
  #Utilizar a banda pixel_qa
  qa = image.select('pixel_qa')
  #0 significa pixels sem cobertura de núvens
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask) #Aqui nos dividimos por 10000 pois é a Constante de conversão do produto Landsat SR

In [0]:
Cloudness_Landsat=collection.map(maskL8sr)

##E Vamos fazer uma mediana para observarmos tudo em uma só imagem!

Coudness_median=Cloudness_Landsat.median()

In [28]:
# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(Coudness_median, {'bands': ['B4', 'B3', 'B2'],'min':0,'max':3000,'gamma':'1.5'},'LANDSAT8')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

In [34]:
##Vamos olhar um produto derivado das imagens Landsat!
#O NDWI é um índice para identificar o teor de umidade na vegetação, e é calculado através do Infravermelho Próximo e Médio

dataset = (ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDWI')
                  .filterDate('2017-01-01', '2017-12-31'))
colorized = dataset.select('NDWI');
colorizedVis = {
  'min': 0.0,
  'max': 1.0,
  'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'],
}

meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(colorized.median(),colorizedVis,'NBRT')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

## ee.Algorithms.Landsat

Existem no EE alguns algoritmos específicos para Landsat.

### Exemplo, covertendo para radiãncia 



In [42]:
#Uma cena "raw" é uma imagem digital numbers (DNs) representando uma escala de radiância. Isso está diretamente relacionado com o tipo de pixel. (8bits, 16bits..)
#Vejamos esse exemplo onde a gente pega uma imagem raw:

raw = ee.Image('LANDSAT/LC08/C01/T1/LC08_231067_20180701');

#Sensores medem a radiância - que corresponde ao brilho numa determinada direção em relação ao sensor.
#Por isso, as imagens de radiância apresentam "Luz difusa" que é refletida pela atmosphera, pelas núvens e etc.

#Calibramos para radiância

radiance = ee.Algorithms.Landsat.calibratedRadiance(raw);

#Aqui estamos mostrando uma conversão para Toa, que é capaz de desviar dos efeitos das radiações refletidas indesejadas.

toa = ee.Algorithms.Landsat.TOA(raw);

meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(raw,{'bands': ['B4', 'B3', 'B2'], 'min': 6000, 'max': 12000}, 'raw')

meuMap.add_ee_layer(radiance, {'bands': ['B4', 'B3', 'B2'], 'max': 90}, 'radiance')

meuMap.add_ee_layer(toa, {'bands': ['B4', 'B3', 'B2'], 'max': 0.2}, 'toa')


# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

## Gerando uma composição Livre de Nuvens

Continuando com os algorítmos específicos para produtos Landsat, possuímos um destinado a elaborar uma composição sem núvens das imagens!



In [54]:
collection = (ee.ImageCollection('LANDSAT/LT05/C01/T1')
    .filterDate('2010-01-01', '2010-12-31'))

#Composição Simples sem escolher parametros
composite = ee.Algorithms.Landsat.simpleComposite(collection);

#Composicao customizando parametros
customComposite = ee.Algorithms.Landsat.simpleComposite(
  collection= collection,
  percentile= 20, #The percentile value to use when compositing each band.(default 50)
  cloudScoreRange= 1 #The size of the range of cloud scores to accept per pixel (default =10)
);

meuMap = folium.Map(location=[20, 0], zoom_start=3, height=800)

folium.Map.add_ee_layer = add_ee_layer

# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(composite, {'bands': ['B3', 'B2', 'B1'], 'max': 128}, 'TOA composite')
meuMap.add_ee_layer(customComposite, {'bands': ['B3', 'B2', 'B1'], 'max': 128},'Custom TOA composite')


# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

## Anexo:

![Imagem_Radiacao](http://un-spider.org/sites/default/files/TOA.JPG)